In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = 'https://www.investing.com/commodities/lean-hogs-historical-data'
browser.visit(url)

In [4]:
# Iterate through all pages
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
#tabletext = soup.find("table").get_text()
texts = soup.find_all("table")
for item in texts:
    if "Open" in item.get_text():
        tabletext = item.get_text()

In [5]:
tabletext

'\n\n\nDate\nPrice\nOpen\nHigh\nLow\nVol. Change %\n\n\n\n\nMay 29, 2020\n56.85\n56.45\n56.92\n55.92\n- -0.13%\n\n\nMay 28, 2020\n56.92\n58.67\n58.80\n56.65\n7.60K -5.40%\n\n\nMay 27, 2020\n60.17\n60.10\n62.00\n60.05\n6.53K -0.54%\n\n\nMay 26, 2020\n60.50\n59.30\n61.30\n58.80\n6.86K 2.93%\n\n\nMay 22, 2020\n58.78\n59.03\n59.72\n58.55\n5.04K -0.97%\n\n\nMay 21, 2020\n59.35\n57.03\n59.42\n56.95\n8.18K 4.35%\n\n\nMay 20, 2020\n56.88\n56.05\n57.05\n55.75\n5.81K 0.40%\n\n\nMay 19, 2020\n56.65\n57.60\n58.65\n56.55\n6.19K -1.73%\n\n\nMay 18, 2020\n57.65\n58.50\n58.80\n57.10\n9.45K -0.39%\n\n\nMay 15, 2020\n57.88\n58.22\n59.40\n57.75\n11.61K -15.73%\n\n\nMay 14, 2020\n68.68\n68.25\n68.75\n67.90\n0.24K 0.99%\n\n\nMay 13, 2020\n68.00\n67.50\n68.28\n67.25\n0.71K -0.04%\n\n\nMay 12, 2020\n68.03\n67.57\n68.07\n67.32\n0.46K 2.56%\n\n\nMay 11, 2020\n66.32\n67.10\n67.75\n66.22\n1.16K -1.45%\n\n\nMay 08, 2020\n67.30\n68.25\n68.70\n67.20\n0.49K -2.18%\n\n\nMay 07, 2020\n68.80\n67.93\n68.95\n65.55\n0.80K

In [6]:
tabletext = tabletext.replace("\n", "|")
tabletext = tabletext.replace("|-", "|0.00K")
tabletext = tabletext.replace("Vol.", "Volume|")
tabletext = tabletext.replace("K", "K|")
tabletext = tabletext.replace("%", "PCT")
tabletext = tabletext.replace(" Change", "Change")
tabletext

'|||Date|Price|Open|High|Low|Volume|Change PCT|||||May 29, 2020|56.85|56.45|56.92|55.92|0.00K| -0.13PCT|||May 28, 2020|56.92|58.67|58.80|56.65|7.60K| -5.40PCT|||May 27, 2020|60.17|60.10|62.00|60.05|6.53K| -0.54PCT|||May 26, 2020|60.50|59.30|61.30|58.80|6.86K| 2.93PCT|||May 22, 2020|58.78|59.03|59.72|58.55|5.04K| -0.97PCT|||May 21, 2020|59.35|57.03|59.42|56.95|8.18K| 4.35PCT|||May 20, 2020|56.88|56.05|57.05|55.75|5.81K| 0.40PCT|||May 19, 2020|56.65|57.60|58.65|56.55|6.19K| -1.73PCT|||May 18, 2020|57.65|58.50|58.80|57.10|9.45K| -0.39PCT|||May 15, 2020|57.88|58.22|59.40|57.75|11.61K| -15.73PCT|||May 14, 2020|68.68|68.25|68.75|67.90|0.24K| 0.99PCT|||May 13, 2020|68.00|67.50|68.28|67.25|0.71K| -0.04PCT|||May 12, 2020|68.03|67.57|68.07|67.32|0.46K| 2.56PCT|||May 11, 2020|66.32|67.10|67.75|66.22|1.16K| -1.45PCT|||May 08, 2020|67.30|68.25|68.70|67.20|0.49K| -2.18PCT|||May 07, 2020|68.80|67.93|68.95|65.55|0.80K| 1.03PCT|||May 06, 2020|68.10|66.97|68.20|66.50|0.77K| 2.95PCT|||May 05, 2020|66.15|

In [7]:
tablelist = tabletext.split("|||")
tablelist

['',
 'Date|Price|Open|High|Low|Volume|Change PCT',
 '||May 29, 2020|56.85|56.45|56.92|55.92|0.00K| -0.13PCT',
 'May 28, 2020|56.92|58.67|58.80|56.65|7.60K| -5.40PCT',
 'May 27, 2020|60.17|60.10|62.00|60.05|6.53K| -0.54PCT',
 'May 26, 2020|60.50|59.30|61.30|58.80|6.86K| 2.93PCT',
 'May 22, 2020|58.78|59.03|59.72|58.55|5.04K| -0.97PCT',
 'May 21, 2020|59.35|57.03|59.42|56.95|8.18K| 4.35PCT',
 'May 20, 2020|56.88|56.05|57.05|55.75|5.81K| 0.40PCT',
 'May 19, 2020|56.65|57.60|58.65|56.55|6.19K| -1.73PCT',
 'May 18, 2020|57.65|58.50|58.80|57.10|9.45K| -0.39PCT',
 'May 15, 2020|57.88|58.22|59.40|57.75|11.61K| -15.73PCT',
 'May 14, 2020|68.68|68.25|68.75|67.90|0.24K| 0.99PCT',
 'May 13, 2020|68.00|67.50|68.28|67.25|0.71K| -0.04PCT',
 'May 12, 2020|68.03|67.57|68.07|67.32|0.46K| 2.56PCT',
 'May 11, 2020|66.32|67.10|67.75|66.22|1.16K| -1.45PCT',
 'May 08, 2020|67.30|68.25|68.70|67.20|0.49K| -2.18PCT',
 'May 07, 2020|68.80|67.93|68.95|65.55|0.80K| 1.03PCT',
 'May 06, 2020|68.10|66.97|68.20|66.50

In [8]:
cleanerlist = []
for item in tablelist:
    if item != "":
        cleanerlist.append(item.replace("||",""))
cleanerlist

['Date|Price|Open|High|Low|Volume|Change PCT',
 'May 29, 2020|56.85|56.45|56.92|55.92|0.00K| -0.13PCT',
 'May 28, 2020|56.92|58.67|58.80|56.65|7.60K| -5.40PCT',
 'May 27, 2020|60.17|60.10|62.00|60.05|6.53K| -0.54PCT',
 'May 26, 2020|60.50|59.30|61.30|58.80|6.86K| 2.93PCT',
 'May 22, 2020|58.78|59.03|59.72|58.55|5.04K| -0.97PCT',
 'May 21, 2020|59.35|57.03|59.42|56.95|8.18K| 4.35PCT',
 'May 20, 2020|56.88|56.05|57.05|55.75|5.81K| 0.40PCT',
 'May 19, 2020|56.65|57.60|58.65|56.55|6.19K| -1.73PCT',
 'May 18, 2020|57.65|58.50|58.80|57.10|9.45K| -0.39PCT',
 'May 15, 2020|57.88|58.22|59.40|57.75|11.61K| -15.73PCT',
 'May 14, 2020|68.68|68.25|68.75|67.90|0.24K| 0.99PCT',
 'May 13, 2020|68.00|67.50|68.28|67.25|0.71K| -0.04PCT',
 'May 12, 2020|68.03|67.57|68.07|67.32|0.46K| 2.56PCT',
 'May 11, 2020|66.32|67.10|67.75|66.22|1.16K| -1.45PCT',
 'May 08, 2020|67.30|68.25|68.70|67.20|0.49K| -2.18PCT',
 'May 07, 2020|68.80|67.93|68.95|65.55|0.80K| 1.03PCT',
 'May 06, 2020|68.10|66.97|68.20|66.50|0.77K|

In [9]:
listolist = []
for item in tablelist:
    listolist.append(item.split("|"))
listolist

[[''],
 ['Date', 'Price', 'Open', 'High', 'Low', 'Volume', 'Change PCT'],
 ['',
  '',
  'May 29, 2020',
  '56.85',
  '56.45',
  '56.92',
  '55.92',
  '0.00K',
  ' -0.13PCT'],
 ['May 28, 2020', '56.92', '58.67', '58.80', '56.65', '7.60K', ' -5.40PCT'],
 ['May 27, 2020', '60.17', '60.10', '62.00', '60.05', '6.53K', ' -0.54PCT'],
 ['May 26, 2020', '60.50', '59.30', '61.30', '58.80', '6.86K', ' 2.93PCT'],
 ['May 22, 2020', '58.78', '59.03', '59.72', '58.55', '5.04K', ' -0.97PCT'],
 ['May 21, 2020', '59.35', '57.03', '59.42', '56.95', '8.18K', ' 4.35PCT'],
 ['May 20, 2020', '56.88', '56.05', '57.05', '55.75', '5.81K', ' 0.40PCT'],
 ['May 19, 2020', '56.65', '57.60', '58.65', '56.55', '6.19K', ' -1.73PCT'],
 ['May 18, 2020', '57.65', '58.50', '58.80', '57.10', '9.45K', ' -0.39PCT'],
 ['May 15, 2020', '57.88', '58.22', '59.40', '57.75', '11.61K', ' -15.73PCT'],
 ['May 14, 2020', '68.68', '68.25', '68.75', '67.90', '0.24K', ' 0.99PCT'],
 ['May 13, 2020', '68.00', '67.50', '68.28', '67.25', '0.

In [22]:
#Removing blank lists
listolist = [x for x in listolist if x != ['']]
listolist

[['Date', 'Price', 'Open', 'High', 'Low', 'Volume', 'Change PCT'],
 ['',
  '',
  'May 29, 2020',
  '56.85',
  '56.45',
  '56.92',
  '55.92',
  '0.00K',
  ' -0.13PCT'],
 ['May 28, 2020', '56.92', '58.67', '58.80', '56.65', '7.60K', ' -5.40PCT'],
 ['May 27, 2020', '60.17', '60.10', '62.00', '60.05', '6.53K', ' -0.54PCT'],
 ['May 26, 2020', '60.50', '59.30', '61.30', '58.80', '6.86K', ' 2.93PCT'],
 ['May 22, 2020', '58.78', '59.03', '59.72', '58.55', '5.04K', ' -0.97PCT'],
 ['May 21, 2020', '59.35', '57.03', '59.42', '56.95', '8.18K', ' 4.35PCT'],
 ['May 20, 2020', '56.88', '56.05', '57.05', '55.75', '5.81K', ' 0.40PCT'],
 ['May 19, 2020', '56.65', '57.60', '58.65', '56.55', '6.19K', ' -1.73PCT'],
 ['May 18, 2020', '57.65', '58.50', '58.80', '57.10', '9.45K', ' -0.39PCT'],
 ['May 15, 2020', '57.88', '58.22', '59.40', '57.75', '11.61K', ' -15.73PCT'],
 ['May 14, 2020', '68.68', '68.25', '68.75', '67.90', '0.24K', ' 0.99PCT'],
 ['May 13, 2020', '68.00', '67.50', '68.28', '67.25', '0.71K', '

In [28]:
#Creating a clean list of lists where each list has the blanks ('') removed
clistolist = []
for item in listolist:
    clistolist.append(list(filter(None,item)))
clistolist
for item in clistolist:
    for itemtwo in item:
        itemtwo = itemtwo.strip()

In [49]:
#Turning the clean list of lists into a dataframe
df = pd.DataFrame(clistolist[1:], columns = clistolist[0])
df

,Date,Price,Open,High,Low,Volume,Change PCT
0,"May 29, 2020",56.85,56.45,56.92,55.92,0.00K,-0.13PCT
1,"May 28, 2020",56.92,58.67,58.80,56.65,7.60K,-5.40PCT
2,"May 27, 2020",60.17,60.10,62.00,60.05,6.53K,-0.54PCT
3,"May 26, 2020",60.50,59.30,61.30,58.80,6.86K,2.93PCT
4,"May 22, 2020",58.78,59.03,59.72,58.55,5.04K,-0.97PCT
5,"May 21, 2020",59.35,57.03,59.42,56.95,8.18K,4.35PCT
6,"May 20, 2020",56.88,56.05,57.05,55.75,5.81K,0.40PCT
7,"May 19, 2020",56.65,57.60,58.65,56.55,6.19K,-1.73PCT
8,"May 18, 2020",57.65,58.50,58.80,57.10,9.45K,-0.39PCT
9,"May 15, 2020",57.88,58.22,59.40,57.75,11.61K,-15.73PCT


In [50]:
df['Volume'] = df['Volume'].str.replace('K','')
df['Date'] = df['Date'].str.replace(", ","-")
df['Date'] = df['Date'].str.replace("Jan ","01-")
df['Date'] = df['Date'].str.replace("Feb ","02-")
df['Date'] = df['Date'].str.replace("Mar ","03-")
df['Date'] = df['Date'].str.replace("Apr ","04-")
df['Date'] = df['Date'].str.replace("May ","05-")
df['Date'] = df['Date'].str.replace("Jun ","06-")
df['Date'] = df['Date'].str.replace("Jul ","07-")
df['Date'] = df['Date'].str.replace("Aug ","08-")
df['Date'] = df['Date'].str.replace("Sep ","09-")
df['Date'] = df['Date'].str.replace("Oct ","10-")
df['Date'] = df['Date'].str.replace("Nov ","11-")
df['Date'] = df['Date'].str.replace("Dec ","12-")
df['Change PCT'] = df['Change PCT'].str.replace("PCT","")
df['Change PCT'] = pd.to_numeric(df['Change PCT']) / 100
df['Volume'] = pd.to_numeric(df['Volume']) * 1000
df['Price'] = pd.to_numeric(df['Price'])
df['Open'] = pd.to_numeric(df['Open'])
df['High'] = pd.to_numeric(df['High'])
df['Low'] = pd.to_numeric(df['Low'])
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Price,Open,High,Low,Volume,Change PCT
0,2020-05-29,56.85,56.45,56.92,55.92,0.0,-0.0013
1,2020-05-28,56.92,58.67,58.80,56.65,7600.0,-0.0540
2,2020-05-27,60.17,60.10,62.00,60.05,6530.0,-0.0054
3,2020-05-26,60.50,59.30,61.30,58.80,6860.0,0.0293
4,2020-05-22,58.78,59.03,59.72,58.55,5040.0,-0.0097
5,2020-05-21,59.35,57.03,59.42,56.95,8180.0,0.0435
6,2020-05-20,56.88,56.05,57.05,55.75,5810.0,0.0040
7,2020-05-19,56.65,57.60,58.65,56.55,6190.0,-0.0173
8,2020-05-18,57.65,58.50,58.80,57.10,9450.0,-0.0039
9,2020-05-15,57.88,58.22,59.40,57.75,11610.0,-0.1573


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        21 non-null     datetime64[ns]
 1   Price       21 non-null     float64       
 2   Open        21 non-null     float64       
 3   High        21 non-null     float64       
 4   Low         21 non-null     float64       
 5   Volume      21 non-null     float64       
 6   Change PCT  21 non-null     float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 1.3 KB


In [52]:
# Connect to Postgres
rds_connection_string = f"postgres:postgres@localhost:5432/bacon_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [54]:
# Pulling the historical data if it exists
try:
    olddf = pd.read_sql_table("lean_hog_commodity", con=engine)
    fulldf = olddf.append(df,ignore_index=True)
except:
    fulldf = df
fulldf = fulldf.drop_duplicates('Date',keep='last')
fulldf

,Date,Price,Open,High,Low,Volume,Change PCT
1,"May 27, 2020",60.17,60.10,62.00,60.05,6.53K,-0.54PCT
2,"May 26, 2020",60.50,59.30,61.30,58.80,6.86K,2.93PCT
3,"May 22, 2020",58.78,59.03,59.72,58.55,5.04K,-0.97PCT
4,"May 21, 2020",59.35,57.03,59.42,56.95,8.18K,4.35PCT
5,"May 20, 2020",56.88,56.05,57.05,55.75,5.81K,0.40PCT
6,"May 19, 2020",56.65,57.60,58.65,56.55,6.19K,-1.73PCT
7,"May 18, 2020",57.65,58.50,58.80,57.10,9.45K,-0.39PCT
8,"May 15, 2020",57.88,58.22,59.40,57.75,11.61K,-15.73PCT
9,"May 14, 2020",68.68,68.25,68.75,67.90,0.24K,0.99PCT
10,"May 13, 2020",68.00,67.50,68.28,67.25,0.71K,-0.04PCT


In [55]:
# Load Lean Hog Commodity to Postgres
fulldf.to_sql(name='lean_hog_commodity', con = engine, if_exists='replace', index = False)